<a href="https://colab.research.google.com/github/elangbijak4/Time-Series-Research/blob/main/Modifikasi1_TSMIXER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perubahan dari TSMIXER, adalah penambahan mekanisme attention dan lapisan dropout menjadi spatialdroput1D. Pertama untuk mempertajam penyerapan pola, kedua untuk mengurangi mekanisme drop neuron yang berlebihan.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

def res_block(inputs, norm_type, activation, dropout, ff_dim):
    """Residual block of Enhanced TSMixer."""
    norm = (
        layers.LayerNormalization
        if norm_type == 'L'
        else layers.BatchNormalization
    )

    # Temporal Linear with Attention
    x = norm(axis=[-2, -1])(inputs)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.MultiHeadAttention(num_heads=4, key_dim=x.shape[-1])(x, x)
    x = layers.Dense(x.shape[-1], activation=activation)(x)
    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Input Length, Channel]
    x = layers.SpatialDropout1D(dropout)(x)
    res = x + inputs

    # Feature Linear
    x = norm(axis=[-2, -1])(res)
    x = layers.Dense(ff_dim, activation=activation)(
        x
    )  # [Batch, Input Length, FF_Dim]
    x = layers.SpatialDropout1D(dropout)(x)
    x = layers.Dense(inputs.shape[-1])(x)  # [Batch, Input Length, Channel]
    x = layers.SpatialDropout1D(dropout)(x)
    return x + res

def build_enhanced_model(input_shape, pred_len, norm_type, activation, n_block, dropout, ff_dim, target_slice):
    """Build Enhanced TSMixer model."""
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs  # [Batch, Input Length, Channel]
    for _ in range(n_block):
        x = res_block(x, norm_type, activation, dropout, ff_dim)

    if target_slice:
        x = x[:, :, target_slice]

    x = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Channel, Input Length]
    x = layers.Dense(pred_len)(x)  # [Batch, Channel, Output Length]
    outputs = tf.transpose(x, perm=[0, 2, 1])  # [Batch, Output Length, Channel])

    return tf.keras.Model(inputs, outputs)